In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
dir = ""

In [3]:
match_df = pd.read_csv(dir + "match_clean.csv")

match_df = match_df.drop('Unnamed: 0',axis=1)

In [4]:
match_df = match_df.sort_values(["league_id", "date", "match_id"])

In [5]:
rating_df = match_df[["league_id", "season", "date", "stage", "match_id", "home_team_id",
                      "away_team_id", "home_team_goal", "away_team_goal"]]

rating_df["score_home_v_away"] = rating_df["home_team_goal"] - rating_df["away_team_goal"]

rating_df["score_away_v_home"] = rating_df["away_team_goal"] - rating_df["home_team_goal"]

rating_df.loc[(rating_df["league_id"]==10257) & (rating_df["season"]=="2014/2015"), "stage"] = 38

rating_df["home_m_rating"] = 0

rating_df["away_m_rating"] = 0

<ipython-input-5-8a2762457f41>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df["score_home_v_away"] = rating_df["home_team_goal"] - rating_df["away_team_goal"]
<ipython-input-5-8a2762457f41>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df["score_away_v_home"] = rating_df["away_team_goal"] - rating_df["home_team_goal"]
/Users/Watanatornpitakp/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [10]:
abs(rating_df["home_team_goal"]-rating_df["away_team_goal"]).value_counts().sort_index() /len(rating_df)

0.0     0.253936
1.0     0.369067
2.0     0.221112
3.0     0.097366
4.0     0.040563
5.0     0.012199
6.0     0.004041
7.0     0.001182
8.0     0.000419
9.0     0.000076
10.0    0.000038
dtype: float64

In [6]:
for league in rating_df["league_id"].unique(): 
    
    print(league)
    
    for season in rating_df["season"].unique():
        
        ls_df = rating_df.loc[(rating_df["league_id"]==league) & (rating_df["season"]==season)]
        
        ls_teams = set(ls_df["home_team_id"].tolist() + ls_df["away_team_id"].tolist())
        
        rating_dict = {}
        record_dict = {}
        
        for team in ls_teams:
            rating_dict[team] = [0]
            record_dict[team] = []

        for stage in range(1, int(max(rating_df["stage"]) +1)):

            lss_df = ls_df.loc[(ls_df["stage"]==stage)]
            lss_df = lss_df.sort_values(["date", "match_id"])


            for home, away in zip(lss_df["home_team_id"], lss_df["away_team_id"]):
                record_dict[home] = record_dict[home] + [away]
                record_dict[away] = record_dict[away] + [home]

            for home_team, home_spread, away_team, away_spread, date in zip(lss_df["home_team_id"], 
                                                                            lss_df["score_home_v_away"],
                                                                            lss_df["away_team_id"], 
                                                                            lss_df["score_away_v_home"],
                                                                            lss_df["date"]):

                home_spread = min(5, home_spread)
                away_spread = min(5, away_spread)

                home_matches = len(record_dict[home_team])
                home_new_rating = ((home_matches-1)/home_matches)*rating_dict[home_team][-1]
                home_new_rating += ((home_spread + rating_dict[away_team][-1])/home_matches)

                away_matches = len(record_dict[away_team])
                away_new_rating = ((away_matches-1)/away_matches)*rating_dict[away_team][-1]
                away_new_rating += ((away_spread + rating_dict[home_team][-1])/away_matches)

                rating_dict[home_team] += [home_new_rating]
                rating_dict[away_team] += [away_new_rating]

                rating_df.loc[(rating_df["home_team_id"]==home_team) &
                              (rating_df["date"]==date), "home_m_rating"] = home_new_rating
                
                rating_df.loc[(rating_df["away_team_id"]==away_team) &
                              (rating_df["date"]==date), "away_m_rating"] = away_new_rating

1
1729
4769
7809
10257
13274
15722
17642
19694
21518
24558


In [7]:
rating_hist_df = pd.concat([pd.DataFrame(zip(rating_df["date"], 
                                             rating_df["home_team_id"], 
                                             rating_df["home_m_rating"])),
                            pd.DataFrame(zip(rating_df["date"], 
                                             rating_df["away_team_id"], 
                                             rating_df["away_m_rating"]))])

rating_hist_df.columns=["date", "team_id", "m_rating"]

rating_hist_df = rating_hist_df.sort_values(["team_id","date"])

rating_hist_df["current_m_rating"] = rating_hist_df.groupby("team_id")["m_rating"].shift(1)

rating_hist_df["current_m_rating"] = rating_hist_df["current_m_rating"].replace(np.nan, 0)

In [8]:
rating_df.columns

Index(['league_id', 'season', 'date', 'stage', 'match_id', 'home_team_id',
       'away_team_id', 'home_team_goal', 'away_team_goal', 'score_home_v_away',
       'score_away_v_home', 'home_m_rating', 'away_m_rating'],
      dtype='object')

In [9]:
home_df = rating_df[['league_id', 'season', 'date', 'stage', 'match_id', 
                     'home_team_id', 'home_team_goal', 'home_m_rating']]

home_df.columns = ['league_id', 'season', 'date', 'stage', 'match_id', 
                   'team_id', 'goal', 'm_rating']

home_df["home"] = 1

away_df = rating_df[['league_id', 'season', 'date', 'stage', 'match_id', 
                     'away_team_id', 'away_team_goal', 'away_m_rating']]

away_df.columns = ['league_id', 'season', 'date', 'stage', 'match_id', 
                   'team_id', 'goal', 'm_rating']

away_df["home"] = 0

rating_df = pd.concat([home_df, away_df], axis=0)

<ipython-input-9-c5dd843a8d61>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_df["home"] = 1
<ipython-input-9-c5dd843a8d61>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_df["home"] = 0


In [10]:
rating_df = rating_df.drop("m_rating", axis=1).merge(rating_hist_df, on=["date", "team_id"], how="left")

In [11]:
rating_df = rating_df.sort_values(["league_id", "match_id", "home"])

In [12]:
rating_df.head()

,league_id,season,date,stage,match_id,team_id,goal,home,m_rating,current_m_rating
26237,1,2008/2009,2008-08-17,1.0,492473,9993,1.0,0,0.0,0.0
6,1,2008/2009,2008-08-17,1.0,492473,9987,1.0,1,0.0,0.0
26231,1,2008/2009,2008-08-16,1.0,492474,9994,0.0,0,0.0,0.0
0,1,2008/2009,2008-08-16,1.0,492474,10000,0.0,1,0.0,0.0
26232,1,2008/2009,2008-08-16,1.0,492475,8635,3.0,0,3.0,0.0


In [13]:
rating_df.to_csv(dir + "rating_df.csv")